## **Postulación Sunai: Python Developer JR**

In [1]:
# Load packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('default')
%matplotlib qt

In [ ]:
#Set working directory
# path_in = input('Enter path to working directory: ')
path_in = r'C:\Users\fespi\OneDrive\Desktop\postulacion_sunai'

files = [os.path.join(path_in, f) for f in os.listdir(path_in) if f.endswith('.xlsx')]

In [ ]:
# Read data and transform to dataframe
def data_sunai(files):
    df = pd.read_excel(files, header=0, parse_dates=True, na_filter=True)
    df.replace('data_faltante',0, inplace=True)
    df['date'], df['time'] = zip(*[(d.date(), d.time()) for d in df['fecha_im']])
    df['date'], df['time'] = df['date'].astype(str), df['time'].astype(str)
    cols = list(df.columns.values)
    df = df[cols[0:2] + cols[-2:] + cols[2:-2]]
    df = df[df.columns[[0,2,3,14]]]
    
    df_pv = df.pivot(index='time', columns='id_i', values='active_power_im')  # Create a pivot table
    df_pv.rename_axis(None, axis=1, inplace=True)
    
    df_inv2 = pd.DataFrame(index = ['sum', 'min', 'max'], columns = df_pv.columns, 
                      data = [df_pv.sum().values, df_pv.min().values, df_pv.max().values]).T
    df_inv2.fillna(0, inplace=True)  # To replace NaN values with 0
    df_inv2.index.name = 'id_i'
    return df_inv2

In [ ]:
for i in range(len(files)):
    data_sunai(files[i])